In this notebook we are trying to simulate the results obtained by lab experiment in the paper Improvung Investment Decisonjs with simulated experience by Bradbury et. al 2015

The authors constructed an index of return distribution. They calculated empirical 1-year returns over 20 years until 08/24/2011 using a 1-day rolling time window, which resulted in 5,042 yearlyoverlapping returns.

We begin by trying to replicate the index.

In [ ]:
!conda update --all

In [1]:
# importing the necessary libraries
import pandas as pd
import numpy as np
import yfinance as yf
import random

In [2]:
# creating the Data Frame
df = yf.download('^GSPC', start='1990-08-24', end='2011-08-24');


[*********************100%***********************]  1 of 1 completed


In [3]:
# Creating the empirical return distribution
returns = (df.Open - df.Open.shift(252))/df.Open.shift(252)
distribution = returns.dropna()
print(distribution.mean(),distribution.std())

0.08288017854619155 0.18078483020830713


In [4]:
sorted_dist = pd.DataFrame(distribution.sort_values(ascending=True))

In [5]:
import seaborn as sns
sns.displot(sorted_dist, x="open")

AttributeError: module 'seaborn' has no attribute 'displot'

In [16]:
print(sns.__version__)

0.9.0


In [6]:
# Defining the products
def product_1(distribution): #product 103.6 
    extraction = round(random.choice(distribution)*100,2)
    product_return = 3.6
    
    return extraction,product_return
     
def product_2(distribution):    #product 100
    extraction = round(random.choice(distribution)*100,2)
    if extraction < 0.0:
        product_return = 0.0
    else:
        product_return = round(0.4*extraction,2)
    
    return extraction,product_return

def product_3(distribution):    #product 90
    extraction = round(random.choice(distribution)*100,2)
    if extraction < -13.70:
        product_return = -10.0
    else:
        product_return = round(0.73*extraction,2)
    
    return extraction,product_return

def product_4(distribution):    #product 80
    extraction = round(random.choice(distribution)*100,2)
    if extraction < -22.00:
        product_return = -20.0
    else:
        product_return = round(0.91*extraction,2)
     
    return extraction,product_return

def product_5(distribution):    #product 0
    extraction = round(random.choice(distribution)*100,2)
    product_return = extraction
    
    return extraction,product_return

In [7]:
# Defining the different value functions for each product, given the reference point
#using the original Value function by Khaneman and Tversky:
# V(x)=x^0.88 for gains and V(x) = 2.25x^0.88

def utility(distribution,product):
    product_index = int(product[-1])
    realized_return = globals()[product](distribution)[1]
    ref_point = 3.60 #[3.60, 3.28, 5.986, 7.462, 8.2][product_index-1]
    x = float(round(realized_return-ref_point,4))
    if x == 0.0:
        return 0.0
    elif x > 0.0:
        return x ** 0.88
    else:
        return -2.25 * (-x)**0.88

In [8]:
# Defining the rule by which a person decides to switch product
def new_choice_utility(last_product):
    product_index = int(last_product[-1])
    results = globals()[last_product](distribution)
    utility_p1 = utility(distribution,"product_1")
    utility_p2 = utility(distribution,"product_2")
    utility_p3 = utility(distribution,"product_3")
    utility_p4 = utility(distribution,"product_4")
    utility_p5 = utility(distribution,"product_5") 
    utilities = [utility_p1,utility_p2, utility_p3, utility_p4, utility_p5]
    max_utility = np.argmax(utilities) + 1
    if max(utilities) > utilities[product_index-1] * 1.25:
        return max_utility
    else:    
        return product_index

In [9]:
# Now we will simulate the experiment, assuming a sample of 1_000_000 people, distributed as in figure 3 of the paper
product_1_holders = int(1000000 * 0.067)
product_2_holders = int(1000000 * 0.21)
product_3_holders = int(1000000 * 0.419)
product_4_holders = int(1000000 * 0.267)
product_5_holders = int(1000000 * 0.038)

In [10]:
# Determining the new choices with utilities
new_choice_from_p1 = [new_choice_utility("product_1") for i in range(product_1_holders)]
new_choice_from_p2 = [new_choice_utility("product_2") for i in range(product_2_holders)]
new_choice_from_p3 = [new_choice_utility("product_3") for i in range(product_3_holders)]
new_choice_from_p4 = [new_choice_utility("product_4") for i in range(product_4_holders)]
new_choice_from_p5 = [new_choice_utility("product_5") for i in range(product_5_holders)]

In [11]:
#Aggregating, new distribution from product 1
sum([nc == 1 for nc in new_choice_from_p1]), sum([nc == 2 for nc in new_choice_from_p1]), sum([nc == 3 for nc in new_choice_from_p1]), sum([nc == 4 for nc in new_choice_from_p1]), sum([nc == 5 for nc in new_choice_from_p1])

(765, 4588, 15231, 21625, 24791)

In [12]:
#Aggregating, new distribution from product 2
sum([nc == 1 for nc in new_choice_from_p2]), sum([nc == 2 for nc in new_choice_from_p2]), sum([nc == 3 for nc in new_choice_from_p2]), sum([nc == 4 for nc in new_choice_from_p2]), sum([nc == 5 for nc in new_choice_from_p2])

(2536, 19062, 46385, 65716, 76301)

In [13]:
#Aggregating, new distribution from product 3
sum([nc == 1 for nc in new_choice_from_p3]), sum([nc == 2 for nc in new_choice_from_p3]), sum([nc == 3 for nc in new_choice_from_p3]), sum([nc == 4 for nc in new_choice_from_p3]), sum([nc == 5 for nc in new_choice_from_p3])

(4952, 26035, 118350, 125256, 144407)

In [14]:
#Aggregating, new distribution from product 4
sum([nc == 1 for nc in new_choice_from_p4]), sum([nc == 2 for nc in new_choice_from_p4]), sum([nc == 3 for nc in new_choice_from_p4]), sum([nc == 4 for nc in new_choice_from_p4]), sum([nc == 5 for nc in new_choice_from_p4])

(3229, 16473, 55377, 102252, 89669)

In [15]:
#Aggregating, new distribution from product 5
sum([nc == 1 for nc in new_choice_from_p5]), sum([nc == 2 for nc in new_choice_from_p5]), sum([nc == 3 for nc in new_choice_from_p5]), sum([nc == 4 for nc in new_choice_from_p5]), sum([nc == 5 for nc in new_choice_from_p5])

(454, 2404, 7655, 11173, 16314)

In [776]:
# Simulating the control experiment, starting with more agents holding more riksy products
product_1_holders_c = int(1000000 * 0.019)
product_2_holders_c = int(1000000 * 0.171)
product_3_holders_c = int(1000000 * 0.429)
product_4_holders_c = int(1000000 * 0.276)
product_5_holders_c = int(1000000 * 0.105)

In [777]:
# Determining the new choices with utilities
new_choice_from_p1_c = [new_choice_utility("product_1") for i in range(product_1_holders_c)]
new_choice_from_p2_c = [new_choice_utility("product_2") for i in range(product_2_holders_c)]
new_choice_from_p3_c = [new_choice_utility("product_3") for i in range(product_3_holders_c)]
new_choice_from_p4_c = [new_choice_utility("product_4") for i in range(product_4_holders_c)]
new_choice_from_p5_c = [new_choice_utility("product_5") for i in range(product_5_holders_c)]

In [778]:
#Aggregating, new distribution from product 1
sum([nc == 1 for nc in new_choice_from_p1_c]), sum([nc == 2 for nc in new_choice_from_p1_c]), sum([nc == 3 for nc in new_choice_from_p1_c]), sum([nc == 4 for nc in new_choice_from_p1_c]), sum([nc == 5 for nc in new_choice_from_p1_c])

(637, 2573, 4550, 5411, 5829)

In [779]:
#Aggregating, new distribution from product 2
sum([nc == 1 for nc in new_choice_from_p2_c]), sum([nc == 2 for nc in new_choice_from_p2_c]), sum([nc == 3 for nc in new_choice_from_p2_c]), sum([nc == 4 for nc in new_choice_from_p2_c]), sum([nc == 5 for nc in new_choice_from_p2_c])

(5664, 27630, 38353, 47613, 51740)

In [780]:
#Aggregating, new distribution from product 3
sum([nc == 1 for nc in new_choice_from_p3_c]), sum([nc == 2 for nc in new_choice_from_p3_c]), sum([nc == 3 for nc in new_choice_from_p3_c]), sum([nc == 4 for nc in new_choice_from_p3_c]), sum([nc == 5 for nc in new_choice_from_p3_c])

(14358, 53611, 119694, 115544, 125793)

In [781]:
#Aggregating, new distribution from product 4
sum([nc == 1 for nc in new_choice_from_p4_c]), sum([nc == 2 for nc in new_choice_from_p4_c]), sum([nc == 3 for nc in new_choice_from_p4_c]), sum([nc == 4 for nc in new_choice_from_p4_c]), sum([nc == 5 for nc in new_choice_from_p4_c])

(9232, 34506, 61279, 90787, 80196)

In [782]:
#Aggregating, new distribution from product 5
sum([nc == 1 for nc in new_choice_from_p5_c]), sum([nc == 2 for nc in new_choice_from_p5_c]), sum([nc == 3 for nc in new_choice_from_p5_c]), sum([nc == 4 for nc in new_choice_from_p5_c]), sum([nc == 5 for nc in new_choice_from_p5_c])

(3527, 13203, 23351, 28197, 36722)